# Pipeline avancée

URL : https://youtu.be/41mnga4ptso?si=jgRd3kUSWGb0LbCA

In [3]:
#importations
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, Binarizer
from sklearn.linear_model import SGDClassifier
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline, make_union

In [4]:
#charger le dataset du Titanic
titanic = sns.load_dataset('titanic')

#afficher les 5 premières lignes de titanic
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [5]:
#afficher les infos sur titanic
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [6]:
#définir X
X = titanic.drop('survived', axis=1)

#définir y
y = titanic['survived']

Si on désire traiter le dataset avec différents transformers, comme par exemple StandardScaler, il faut faire le tri dans les différentes colonnes, parce que StandardScaler n'accepte de traiter que les variables numériques (sinon il retourne une erreur).

Mais lorsqu'on a une pipeline, avec à son bout un estimaor, on a envie de faire passer le dataset en entier, afin de pouvoir utiliser toutes les colonnes pour entraîner l'estimator. Mais si on fait ça, le StandardScaler, qui se situe au début de la pipepline va retourner une erreur, puisqu'on va lui demander d'utiliser tout le tableau X (càd toutes les colonnes) pour faire une normalisation. Or, il ne peut pas transformer les colonnes avec des valeurs non numériques (exp: sex, embarked,...etc).

Démonstration:

In [8]:
#créer une pipeline avec StandardScaler() et SGDClassifier()
model = make_pipeline(StandardScaler(), SGDClassifier())

#entrainer le modèle avec la méthode fit() sur les données X et y
model.fit(X, y)

ValueError: could not convert string to float: 'male'

Pour résoudre ce problème, il faut créer dans la pipeline un mécanisme qui permet de trier les colonnes, afin de donner certaines colonnes à certains transformers et d'autres colonnes à d'autres transformers.

Pour faire ça, on utilise la fonction **make_column_transformer()**.

## make_column_transformer

ColumnTransformer permet d'appliquer les transformers sur les colonnes qu'on selectionne:

**make_column_transformer((transformer, ['colonne1', 'colonne2', ...]))**

In [28]:
#créer un transformer de colonnes
transformer = make_column_transformer((StandardScaler(),
                                      ['age', 'fare']))

#entraîner le transformer avec fit_transform()
transformer.fit_transform(X)

array([[-0.53037664, -0.50244517],
       [ 0.57183099,  0.78684529],
       [-0.25482473, -0.48885426],
       ...,
       [        nan, -0.17626324],
       [-0.25482473, -0.04438104],
       [ 0.15850313, -0.49237783]])

In [29]:
#même si on fait passer tout le tableau X, le transformer ne traîte que les 2 colonnes sélectionnées (age et fare)

**Application classique du ColumnTransformer:**

1. Séparer les catégories et variables numériques en **2 listes**.
2. Utliser le ColumnTransformer pour **traiter chaque liste de variables séparément**.

Exemple:

In [31]:
#infos sur titanic
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [32]:
#créer les 2 listes de variables
#liste des variables numériques
numerical_features = ['pclass', 'age', 'fare']

#liste des variables non numériques
categorical_features = ['sex', 'deck', 'alone']

A partir de ces 2 listes de variables, on utilise le ColumnTransformer make_column_transformer(), pour appliquer une 1ère série de transformation sur les variables numériques (numerical_features), puis une 2ème série de transformation sur les variables non numériques (categorical_features):

make_column_transformer((....... , numerical_features),
                        (....... , categorical_features))

Et on doit écrire ces séries de transformation sous forme de pipelines:

**make_column_transformer((numerical_pipeline, numerical_features),
                        (categorical_pipeline, categorical_features))**

In [34]:
#créer numerical_pipeline pour traiter les variables numériques
#SimpleImputer pour enlever les valeurs manquantes
numerical_pipeline = make_pipeline(SimpleImputer(),
                                   StandardScaler())

#créer categorical_pipeline pour traiter les variables non numériques
#SimpleImputer avec strategy= 'most_frequent' càd remplacer les valeurs manquantes par les + fréquentes 
categorical_pipeline = make_pipeline(SimpleImputer(strategy= 'most_frequent'),
                                     OneHotEncoder())

In [35]:
#assembler le tout avec make_column_transformer (tout ça donne un transformer qu'on appelle ici preprocessor)
preprocessor = make_column_transformer((numerical_pipeline, numerical_features),
                                       (categorical_pipeline, categorical_features))

Il ne reste plus qu'à créer la pipeline finale:

In [37]:
#créer la pipeline finale
model = make_pipeline(preprocessor,
                      SGDClassifier())

#entraîner model (la pipeline finale) avec fit() sur les données X et y
model.fit(X, y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['pclass', 'age', 'fare']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  ['sex', 'deck', 'alone'])])),
                ('sgdclassifier', SGDClassifier())])

In [38]:
#le modèle est entraîné!

**Résumé:**

Pour créer des pipelines plus sophistiquées qui permettent de traiter des datasets hétérogènes:

1. Faire le tri des variables numériques (numerical_features) et des variables non numériques (categorical_features).
2. Définir, pour chaque type de variables, une pipeline (une chaîne de transformation que ces variables vont subir): une pipeline numérique (numerical_pipeline) pour les variables numériques (numerical_features), et une pipeline catégorique (categorical_pipeline) pour les variables numériques (categorical_features).
3. Injecter ces 2 pipelines dans la fonction make_column_transformer(), ça retourne un preprocessor - qu'on peut entraîner avec fit() et transform() - qu'on utilise ce preprocessor dans une pipeline finale, celle qui définit le modèle de ML.

## make_column_selector

make_column_selector() permet de faciliter la sélection des colonnes.

On a 3 arguments au choix:
- dtype_include
- dtype_exclude
- pattern (avec des Regular Expressions)

Exemple de types de colonnes:
np.number -> variables numériques
object -> variables non numériques (catégorielles)

In [42]:
#créer les 2 listes de variables
#liste des variables numériques
#make_column_selector sélectionne toutes les variables numériques
numerical_features = make_column_selector(dtype_include=np.number)

#liste des variables non numériques
#make_column_selector sélectionne toutes les variables sauf les variables numériques
categorical_features = make_column_selector(dtype_exclude=np.number)

In [43]:
#créer numerical_pipeline pour traiter les variables numériques
#SimpleImputer pour enlever les valeurs manquantes
numerical_pipeline = make_pipeline(SimpleImputer(),
                                   StandardScaler())

#créer categorical_pipeline pour traiter les variables non numériques
#SimpleImputer avec strategy= 'most_frequent' càd remplacer les valeurs manquantes par les + fréquentes 
categorical_pipeline = make_pipeline(SimpleImputer(strategy= 'most_frequent'),
                                     OneHotEncoder())

In [44]:
#assembler le tout avec make_column_transformer (tout ça donne un transformer qu'on appelle ici preprocessor)
preprocessor = make_column_transformer((numerical_pipeline, numerical_features),
                                       (categorical_pipeline, categorical_features))

In [45]:
#créer la pipeline finale
model = make_pipeline(preprocessor,
                      SGDClassifier())

#entraîner model (la pipeline finale) avec fit() sur les données X et y
model.fit(X, y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000002917B9BC0E0>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000002917A272360>)])),
                ('sgdclassifier', SGDClassifier())])

In [46]:
#on a ici un modèle qui est entraîné avec toutes variables du dataset titanic
#tout à l'heure on sélectionné que quelques colonnes du dataset titanic

Il est maintenant temps de voir les pipelines paralèlles avec make_union().

## make_union

Jusqu'à maintenant, on ne faisait que créer des pipelines en mettant bout à bout plusieurs transformers (une série de transformers). Avec Sklearn, on peut créer des pipelines qui mettent en parallèle plusieurs transformers, et qui rejoignent leurs résultats ensemble dans un seul et même tableau.

Exemple: (on utilise seulement les colonnes age et fare pour simplifier l'explication)

In [50]:
#définir X
X = titanic.drop('survived', axis=1)

#définir y
y = titanic['survived']

In [51]:
#définir numerical_features
numerical_features = X[['age', 'fare']]

#afficher numerical_features
numerical_features

,age,fare
0,22.0,7.2500
1,38.0,71.2833
2,26.0,7.9250
3,35.0,53.1000
4,35.0,8.0500
...,...,...
886,27.0,13.0000
887,19.0,30.0000
888,NaN,23.4500
889,26.0,30.0000


Dans la fonction make_union(), tout ce qu'on a faire c'est d'entrer les transformers qu'on veut avoir en parallèle:

**pipeline = (StandardScaler(), Binarizer())**

Puis on entraîne avec fit_transform() sur les doonées (numerical_features):

**pipeline.fit_transform(numerical_features)**

On obtient un tableau de 4 colonnes: 2 colonnes standardisées avec StandardScaler(), et 2 colonnes discrétisées avec Binarizer()